# Urban Heat Island Analysis Using the Landsat 8 Satellite Data

This script will be based on the proposed methodology of KAPLAN, G., et.al in the journal [Urban Heat Island Analysis Using the Landsat 8 Satellite Data: A Case Study in Skopje, Macedonia](https://www.mdpi.com/2504-3900/2/7/358).

## Import libraries

In [1]:
import folium
import geopandas as gpd
import json
import ee

# Start the earth engine library
ee.Initialize()

C:\Anaconda3\envs\geo\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


## Import the study area

Hermosillo is the capital of the state of Sonora, and is located in the north-western Mexico. The municipality is mostly flat with sloping towards the sea with a population of 936,263 citizents. The municipality covers an area of 14,880.2 km2, and is situated on a median height of 569 m above the sea level. Hermosillo has an average temperature of 25,1 °C, and its average rainfall per year is 386.9 mm

In [2]:
# Import shapefile

# Change the route of the shapefile
# Set the file route
fp = "C:\\Users\\Juan Alexis\\Documents\\QUANTUN\\Urban_Heat_Island\\Study_Area\\Hermosillo_city.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)

In [3]:
# Convert the shapefile to json
aoi = data.to_json()

# load the json file
aoi = json.loads(aoi)

# Select the features
aoi = aoi['features']

# Print the result
# aoi

In [4]:
# Convert the area of interest to FeatureCollection
area = ee.FeatureCollection(aoi).geometry()
area.getInfo()

{'type': 'MultiPolygon',
 'coordinates': [[[[-110.96385260028632, 28.999863372177366],
    [-110.96393191420889, 29.000887427737894],
    [-110.96501119746219, 29.00087488480638],
    [-110.96578162741572, 29.00084484629917],
    [-110.96722794034082, 28.998116890347376],
    [-110.96864733955834, 28.9989293201395],
    [-110.96930168016591, 28.999265250184152],
    [-110.970351510459, 28.997287579978472],
    [-110.97061691003975, 28.996808540363368],
    [-110.97088811971996, 28.996287479790283],
    [-110.96993066013056, 28.99573174974099],
    [-110.96901832007076, 28.99520443030849],
    [-110.96880174958535, 28.995311999750005],
    [-110.96696966006178, 28.995382110345165],
    [-110.96570827987672, 28.9953942902988],
    [-110.96358395007273, 28.99545977998396],
    [-110.9615718961978, 28.995515323729872],
    [-110.95932181817987, 28.99557362107535],
    [-110.9594027204242, 28.997857904277797],
    [-110.95968825538004, 28.99784838629284],
    [-110.96162543377272, 28.997811

## Select the image collection 

Select the collection **USGS Landsat 8 Level 2, Collection 2, Tier 1** 

For more information: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2

The thermal band **ST_B10** is already converted to surface temperature, it only needs to be scaled and clipped to the area of interest.

### Select the time range

For tihs script, we´re going to use a temporal range of all the images form 2021.

In [5]:
# Define time range
startyear = 2021
endyear = 2021

# Set the date in ee date format
start_date = ee.Date.fromYMD(startyear, 1, 1)
end_date = ee.Date.fromYMD(endyear, 12, 31)

### Import the image collection

Import Landsat 8 collection and select the bands B4, B5 and B10

In [6]:
'''
Instructions
- Cloud Masking function
- Import Landsat 8 SR collection
- Create a function to scale the bands
- Filter the collection with the time range
- Select ST_B10 band
- Clip the images with the study area
'''

# Cloud Masking function
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    
    return image.updateMask(mask)

# Import dataset, filter with the time range and create a mean image
L8_SR = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterDate(start_date, end_date)\
            .filterBounds(area)\
            .map(maskL8sr)

# Applies scaling factors.
def applyScaleFactors (image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# Apply the function an create a median image
dataset = L8_SR.map(applyScaleFactors).median().clip(area)

In [7]:
# Print all the dataset
dataset.getInfo()

# The scaled bands have the same names

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [2, 2],
   'origin': [-112, 28],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [2, 2],
   'origin': [-112, 28],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [2, 2],
   'origin': [-112, 28],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -0.2,
    'max': 1.6022125},
   'dimensions': [2, 2],
   'origin': [-112, 28],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'SR_B5',
   'dat

## Calculate LST

Calculate Land Surface Temperature (LST) from Band 10. Convert the surface temperature in Kelvin (°K) to Centigrade (°C)

In [8]:
# Select Thermal Band (ST_B10)
thermal = dataset.select('ST_B10')

# Convert surface temperature to °C
def conversion (image):
    return image.subtract(273.15)

# Apply the function
LST = conversion(thermal)

### Refuce function

Compute the limits of Land Surface Temperature band

In [9]:
# Create the reducer function
def reducer_1 (image):
    reduce = image.reduceRegions(**{
        'collection': area,
        'reducer': ee.Reducer.min().combine(**{
            'reducer2': ee.Reducer.max(),
            'sharedInputs': True}),
        'scale': 30
    })
    
    return reduce


# Apply the function
thermal_minmax = reducer_1(LST)

# Print the result
print(thermal_minmax.getInfo())

{'type': 'FeatureCollection', 'columns': {'max': 'Float<-124.14999999999998, 99.84994070000005>', 'min': 'Float<-124.14999999999998, 99.84994070000005>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-110.96385260028632, 28.999863372177366], [-110.96393191420889, 29.000887427737894], [-110.96501119746219, 29.00087488480638], [-110.96578162741572, 29.00084484629917], [-110.96722794034082, 28.998116890347376], [-110.96864733955834, 28.9989293201395], [-110.96930168016591, 28.999265250184152], [-110.970351510459, 28.997287579978472], [-110.97061691003975, 28.996808540363368], [-110.97088811971996, 28.996287479790283], [-110.96993066013056, 28.99573174974099], [-110.96901832007076, 28.99520443030849], [-110.96880174958535, 28.995311999750005], [-110.96696966006178, 28.995382110345165], [-110.96570827987672, 28.9953942902988], [-110.96358395007273, 28.99545977998396], [-110.9615718961978, 28.995515323729872], [-110.959321

## Generate comparative Indexes

### NDVI: Normalized difference vegetation index

The NDVI is a dimensionless index that describes the difference between visible and near-infrared reflectance of vegetation cover and can be used to estimate the density of green on an area of land

In [10]:
# Create the index using .normalizedDifference()
NDVI = dataset.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

comp_NDVI = reducer_1(NDVI)
print(comp_NDVI.getInfo())

{'type': 'FeatureCollection', 'columns': {'max': 'Float<-1.0, 1.0>', 'min': 'Float<-1.0, 1.0>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-110.96385260028632, 28.999863372177366], [-110.96393191420889, 29.000887427737894], [-110.96501119746219, 29.00087488480638], [-110.96578162741572, 29.00084484629917], [-110.96722794034082, 28.998116890347376], [-110.96864733955834, 28.9989293201395], [-110.96930168016591, 28.999265250184152], [-110.970351510459, 28.997287579978472], [-110.97061691003975, 28.996808540363368], [-110.97088811971996, 28.996287479790283], [-110.96993066013056, 28.99573174974099], [-110.96901832007076, 28.99520443030849], [-110.96880174958535, 28.995311999750005], [-110.96696966006178, 28.995382110345165], [-110.96570827987672, 28.9953942902988], [-110.96358395007273, 28.99545977998396], [-110.9615718961978, 28.995515323729872], [-110.95932181817987, 28.99557362107535], [-110.9594027204242, 28.9978

### NDBI:Normalized Difference Built-up Index

In [11]:
NDBI = dataset.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')

NDBI_comps = reducer_1(NDBI)
print(NDBI_comps.getInfo())

{'type': 'FeatureCollection', 'columns': {'max': 'Float<-1.0, 1.0>', 'min': 'Float<-1.0, 1.0>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-110.96385260028632, 28.999863372177366], [-110.96393191420889, 29.000887427737894], [-110.96501119746219, 29.00087488480638], [-110.96578162741572, 29.00084484629917], [-110.96722794034082, 28.998116890347376], [-110.96864733955834, 28.9989293201395], [-110.96930168016591, 28.999265250184152], [-110.970351510459, 28.997287579978472], [-110.97061691003975, 28.996808540363368], [-110.97088811971996, 28.996287479790283], [-110.96993066013056, 28.99573174974099], [-110.96901832007076, 28.99520443030849], [-110.96880174958535, 28.995311999750005], [-110.96696966006178, 28.995382110345165], [-110.96570827987672, 28.9953942902988], [-110.96358395007273, 28.99545977998396], [-110.9615718961978, 28.995515323729872], [-110.95932181817987, 28.99557362107535], [-110.9594027204242, 28.9978

## Calculate Urban Heat Island (UHI)

Urban Heat Island is calculated using LST with this formula:

$$𝑈𝐻𝐼 = µ + 𝜎/2$$

where:
**µ** is the mean LST value and **𝜎** is the standard derivation of the LST

### Reduce function

In [12]:
# Create the reducer function
def reducer (image):
    reduce = image.reduceRegions(**{
        'collection': area,
        'reducer': ee.Reducer.stdDev().combine(**{
            'reducer2': ee.Reducer.mean(),
            'sharedInputs': True}),
        'scale': 30
    })
    
    return reduce


# Apply the function
uhi_comps = reducer(LST)

# Print the result
print(uhi_comps.getInfo())

{'type': 'FeatureCollection', 'columns': {'mean': 'Float<-124.14999999999998, 99.84994070000005>', 'stdDev': 'Float', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-110.96385260028632, 28.999863372177366], [-110.96393191420889, 29.000887427737894], [-110.96501119746219, 29.00087488480638], [-110.96578162741572, 29.00084484629917], [-110.96722794034082, 28.998116890347376], [-110.96864733955834, 28.9989293201395], [-110.96930168016591, 28.999265250184152], [-110.970351510459, 28.997287579978472], [-110.97061691003975, 28.996808540363368], [-110.97088811971996, 28.996287479790283], [-110.96993066013056, 28.99573174974099], [-110.96901832007076, 28.99520443030849], [-110.96880174958535, 28.995311999750005], [-110.96696966006178, 28.995382110345165], [-110.96570827987672, 28.9953942902988], [-110.96358395007273, 28.99545977998396], [-110.9615718961978, 28.995515323729872], [-110.95932181817987, 28.99557362107535], [-110.

In [13]:
# Extract Values
mean_LST = uhi_comps.aggregate_array('mean').getInfo()
print(mean_LST)

stdDev_LST = uhi_comps.aggregate_array('stdDev').getInfo()
print(stdDev_LST)

mean_LST = ee.Image(mean_LST)
stdDev_LST = ee.Image(stdDev_LST)

# Formula
#UHI = 


[41.586305878377374]
[1.9663884078869382]


In [14]:
# https://gis.stackexchange.com/questions/422033/land-surface-temperature-in-google-earth-engine

In [15]:
# RGB
mapid_1 = dataset.getMapId({'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}) # Mosaico RGB
center = [29.075, -110.958333]
map = folium.Map(location=center, zoom_start=12)

folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                 attr=' Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
                 name= 'Esri aerial image', overlay=True, show=False).add_to(map)

folium.TileLayer(
    tiles=mapid_1['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Color Verdadero',
    show=False,
  ).add_to(map)

# NDVI
mapid_2 = NDVI.getMapId({'palette': 
                         ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
                          '74A901', '66A000', '529400', '3E8601', '207401', '056201',
                          '004C00', '023B01','012E01', '011D01', '011301'],
                         'opacity': 0.8, 'min': -0.5, 'max': 0.7}) # Mosaico NDVI
folium.TileLayer(
    tiles=mapid_2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  )#.add_to(map)

# NDBI
mapid_3 = NDBI.getMapId({'min': -0.4, 'max': 0.4, 'opacity': 0.8, 
                        'palette':['306466', '9cab68', 'cccc66', '9c8448', '6e462c']}) 
folium.TileLayer(
    tiles=mapid_3['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDBI',
  )#.add_to(map)

# LST
mapid_4 = LST.getMapId({'min': 20, 'max':50, 'palette': [
'040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003'
 ], 'opacity': 0.5}) # Mosaico LST 
folium.TileLayer(
    tiles=mapid_4['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='LST',
  ).add_to(map)

map.add_child(folium.LayerControl(position = 'topleft'))

import folium.plugins as plugins

#mouse position
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
plugins.MousePosition(position='bottomright', separator=' | ', prefix="Mouse:",lat_formatter=fmtr, lng_formatter=fmtr).add_to(map)


import branca
import branca.colormap as cm

colormap = cm.LinearColormap(colors=['#040274', '#040281', '#0502a3', '#0502b8', '#0502ce', '#0502e6','#0602ff', '#235cb1', '#307ef3', '#269db1', '#30c8e2', '#32d3ef','#3be285', '#3ff38f', '#86e26f', '#3ae237', '#b5e22e', '#d6e21f','#fff705', '#ffd611', '#ffb613', '#ff8b13', '#ff6e08', '#ff500d','#ff0000', '#de0101', '#c21301', '#a71001', '#911003'],
                             #index=[20, 25, 30, 35, 40, 45, 50],
                             #tick_labels = [0, 10, 100, 150, 250, 900, 100],
                             #scale_width = 800, scale_height = 20,
                             vmin= 20, vmax = 50,
                             #position = 'bottomleft',
                             caption='Surface Temperature (°C)')

map.add_child(colormap)

map

In [16]:
# Guardar mapa en un archivo.
map.save('Hermosillo_LST.html')

https://python-visualization.github.io/branca/_modules/branca/colormap.html#LinearColormap

https://python-visualization.github.io/folium/modules.html